In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("ETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [9

In [2]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame -  no need to "unzip" the gz
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz"
spark.sparkContext.addFile(url)



In [3]:
#Load tab-separated file to DataFrame
kitchen_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Kitchen_v1_00.tsv.gz"), inferSchema=True, sep="\t")
kitchen_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   37000337|R3DT59XH7HXR9K|B00303FI0G|     529320574|Arthur Court Pape...|         Kitchen|          5|            0|          0|   N|                Y|Beautiful. Looks ...|Beautiful.  Looks...| 2015-08-31|
|         US|   15272914|R1LFS11BNASSU8|B00JCZKZN6|     274237558|Olde Thompson Bav...|         Kitchen|          5|    

In [12]:
kitchen_df.count()


4880466

In [4]:
from pyspark.sql import Row

clean_kitchen_df = kitchen_df.dropDuplicates()

clean_kitchen_df.show()




+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24197845|R1002B10PD9ZFM|B000LTOCSG|     630072149|Replacement Filte...|         Kitchen|          5|            0|          0|   N|                Y|          Very Nice!|Very inexpensive,...| 2009-07-20|
|         US|   17990743|R1007DQP1BG5WJ|B007M2OHEY|     362955205|Fred & Friends E....|         Kitchen|          5|    

In [18]:
clean_kitchen_df.count()


4880466

In [5]:
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType

In [6]:
# Create dataframe to match "review_id_table" table
user_review2_df = clean_kitchen_df.select(["review_id", "customer_id", "product_id", "product_parent","review_date"])
user_review2_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [7]:
user_review_df = user_review2_df.withColumn("review_date", col("review_date").cast(DateType()))
user_review_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1002B10PD9ZFM|   24197845|B000LTOCSG|     630072149| 2009-07-20|
|R1007DQP1BG5WJ|   17990743|B007M2OHEY|     362955205| 2014-09-12|
|R100AIJ0ZFN8CZ|   33816354|B0001ZYYQU|     331822491| 2007-03-09|
|R100C48B994FWD|   13231777|B00IK59T1S|     744019201| 2014-11-26|
|R100E300CKVFYD|   53006884|B00833DQCU|     298219798| 2013-12-08|
|R100GBKBV767AA|   14608024|B0000CDVD2|     184834728| 2014-04-30|
|R100I1XICZFNHD|   46627762|B0014NKJTO|      21858491| 2014-03-04|
|R100KXBN0DJ74A|   42223395|B000P9GWFS|     826643513| 2012-07-04|
|R100VC4OCYFFWM|   29497970|B0016KXAD2|     641397529| 2015-08-26|
|R100YFR4KUB3GC|   13368891|B003LYVXJ4|     510828298| 2013-12-08|
|R1011FF2WI90BO|   38569985|B000A7W4YS|     719164168| 2005-12-15|
|R101KSJLZDFCG2|   19893443|B0085MQPSG|     839082593| 2014-07

In [8]:
user_review_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [9]:
#Drop Duplicates of key column
clean_review_df = user_review_df.dropDuplicates((["review_id"]))


In [28]:
clean_review_df.count()

4880466

In [10]:
# Create dataframe to match "products" table
products_df = clean_kitchen_df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B000LTOCSG|Replacement Filte...|
|B007M2OHEY|Fred & Friends E....|
|B0001ZYYQU|KitchenAid Artisa...|
|B00IK59T1S|PUR 7 Cup Pitcher...|
|B00833DQCU|Prepworks from Pr...|
|B0000CDVD2|Oxo Good Grips Co...|
|B0014NKJTO|9 Inch Vinyl Sede...|
|B000P9GWFS|KitchenAid Profes...|
|B0016KXAD2|          Party Hats|
|B003LYVXJ4|Aluminum Wire Sof...|
|B000A7W4YS|Braun Tassimo TA ...|
|B0085MQPSG|BigMouth Inc The ...|
|B000H8E03I|Graviti Electroni...|
|B003F6GKZU|Chef'n PepperBall...|
|B000KESQ1G|Ateco Professiona...|
|B00004S7V7|Microplane 40001 ...|
|B004QGXNT4|Kitchen Elements ...|
|B0007SXIMM|KitchenAid KFP740...|
|B0006MEI5A|Oxi Clean Stain R...|
|B0000DE7JR|Stainless Steel R...|
+----------+--------------------+
only showing top 20 rows



In [30]:
products_df.count()

4880466

In [11]:
#Drop Duplicates of key column
clean_products_df = products_df.dropDuplicates((["product_id"]))

In [32]:
clean_products_df.count()

433478

In [12]:
clean_products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [14]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import count

In [15]:
# Create dataframe to match "customers" table
#GroupBy on key column
customers_df = clean_kitchen_df.groupBy("customer_id").agg(count("customer_id").alias("customer_count"))


In [42]:
customers_df.show()


+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   16308995|             2|
|    4709446|             1|
|   31546122|             2|
|   25866981|             7|
|   11537699|             4|
|   50481713|             2|
|   14863908|             2|
|   52964170|             1|
|   51863594|             1|
|   42176175|             5|
|   39258095|             1|
|   52456816|             5|
|   23141101|             3|
|   40302623|             2|
|   18614361|             1|
|   25748567|             2|
|   15185969|             9|
|   30414919|             8|
|   37589671|             1|
|   17018430|             1|
+-----------+--------------+
only showing top 20 rows



In [43]:
customers_df.count()


2830483

In [16]:
# Create dataframe to match "vine_table" table
vine_df = clean_kitchen_df.select(["review_id", "star_rating", "helpful_votes","total_votes", "vine"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
|R1002B10PD9ZFM|          5|            0|          0|   N|
|R1007DQP1BG5WJ|          5|            0|          0|   N|
|R100AIJ0ZFN8CZ|          4|            3|          3|   N|
|R100C48B994FWD|          5|            0|          0|   N|
|R100E300CKVFYD|          1|            3|          3|   N|
|R100GBKBV767AA|          5|            0|          0|   N|
|R100I1XICZFNHD|          1|            1|          1|   N|
|R100KXBN0DJ74A|          1|            4|          6|   N|
|R100VC4OCYFFWM|          5|            0|          0|   N|
|R100YFR4KUB3GC|          1|            0|          1|   N|
|R1011FF2WI90BO|          5|            9|         10|   N|
|R101KSJLZDFCG2|          4|            0|          1|   N|
|R101VMZSKC6RM7|          3|            8|          8|   N|
|R101W7KWZ60CSV|          5|            

In [45]:
vine_df.count()

4880466

In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.ct0o6q0pbonx.us-east-2.rds.amazonaws.com:5432/kitchen_reviews_aws"
config = {"user":"postgres", 
          "password": "Peop1eS0ft", 
          "driver":"org.postgresql.Driver"}

In [76]:
# Append DataFrame to review_id_table table in RDS

clean_review_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [77]:
# Write dataframe to products table in RDS

clean_products_df.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [19]:
# Write dataframe to customers table in RDS

customers_df.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [20]:
# Write dataframe to vine_table table in RDS

vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)